Harrison DPLL. Add splitting rule to DP.

Harrison Iterative.


The concept of a trail. It is often a required judo move to remember that your language is maintaining an explicit stack for you and refiying it as a manipulable object. Odds are your language's stack management does not make it efficient or convenient to do to go beyond simple depth first search.





In [7]:
from typing import Optional
from enum import Enum
from dataclasses import dataclass
Lit = int
Trail = tuple[list[Lit], list[int]] # implicit model in trail, second list is decision points. Instead of tagged list[tuple[List, bool]] trail.
Clause = frozenset[Lit]



def unassigned(trail: Trail, cls : Clause) -> set[Lit]:
    return set(abs(l) for l in cls) - set(abs(l) for l in trail[0])

def unit_prop(trail, cnf) -> Optional[Lit]:
    for cls in cnf:
        unassigned_cls = unassigned(trail, cls)
        if len(unassigned_cls) == 1:
            return next(iter(unassigned_cls))
    return None

Truthy = Enum("Truthy", "SAT UNSAT UNKNOWN")

def truthiness(trail, cnf) -> Truthy:
    pmodel = {**{l : True for l in trail[0] }, **{-l : False for l in trail[0]}} 
    if all(any(pmodel.get(l, False) for l in cls) for cls in cnf):
        return Truthy.SAT
    elif any(all(not pmodel.get(l, True) for l in cls) for cls in cnf):
        return Truthy.UNSAT # given current trail, we can't satisfy the cnf
    else:
        return Truthy.UNKNOWN

def backtrack(trail):
    last_decision_index = trail[1].pop()
    last_decision = trail[0][last_decision_index]
    while len(trail[0]) > last_decision_index:
        trail[0].pop()
    return last_decision

def dpll(cnf):
    trail = ([], [])
    for i in range(100):
        print(trail)
        unit_prop(cnf,trail)
        match truthiness(trail, cnf):
            case Truthy.SAT:
                return Truthy.SAT, trail
            case Truthy.UNSAT:
                # backtrack
                print("backtrack")
                l = backtrack(trail)
                while l < 0:
                    if len(trail[0]) == 0:
                        return Truthy.UNSAT, None
                    l = backtrack(trail)
                trail[0].append(-l)
                trail[1].append(len(trail[0])-1)
            case Truthy.UNKNOWN:
                # decide
                unassigned_lits = set(abs(l) for cls in cnf for l in cls) - set(abs(l) for l in trail[0])

                l = next(iter(unassigned_lits)) # is this okay?
                trail[0].append(abs(l))
                trail[1].append(len(trail[0]) - 1)

#dpll([[1,2],[-2]])
#dpll([[1,2],[-2], [-1]])
dpll([[1,2,3],[-2], [-1], [-1]])


([], [])
([1], [0])
backtrack
([-1], [0])
([-1, 2], [0, 1])
backtrack
([-1, -2], [0, 1])
([-1, -2, 3], [0, 1, 2])


(<Truthy.SAT: 1>, ([-1, -2, 3], [0, 1, 2]))

In [ ]:
@dataclass
class Trail():
    lits : list[Lit]
    decision_points : list[int]
    model : dict[Lit, Optional[bool]] # could be a list.
    def pop(self): # backtrack
        last_decision_index = self.decision_points.pop()
        # TODO: reset model
        last_decision = self.lits[last_decision_index]
    
        return last_decision
    def decide(self, l : Lit):
        self.decision_points.append(len(self.lits))
        self.prop(l)
    def prop(self,  l : Lit):
        self.lits.append(l)
        self.model[abs(l)] = l > 0

# What is the disction between trail and solver. I guess solver has the cnf. two watched literal, etc.


# Modelling
Maybe I need more infrastructure for modeling before I get into the nuts and bolts. Start with the brute force solver, show how it fails on problems of interest. That's a nice narrative.
BitVectors.
BitBlasting
Finitedomains

Use z3 or pysat

Hmm. Maybe this is why harrison starts with cnf encoding.

In [17]:
from dataclasses import dataclass
cnf = []
def clear_cnf():
    global cnf
    cnf = []

@dataclass
class Var():
    name : str
    counter = [0]
    def __init__(self, name):
        self.name = name
        self.idx = self.counter[0]
        self.counter[0] += 1 #?
    def __add__(self, other):
        cnf.append(yada)
        z = Var("z")
    def __eq__(self, other):
        res = Var("res")
        cnf.append([res, -self.idx, -other.idx])  # other & self => res
        cnf.append([-res, self.idx, -other.idx])  
        cnf.append([-res, -self.idx, other.idx]) 
        cnf.append([res, self.idx, other.idx])   # not other & not self => res
        return res
    def __and__(self, other):
    def __not__(self):
        res = Var("~" + self.name)
        res.idx = -self.idx
        return res


    def __or__(self, other):
        res  = Var("res")
        cnf.append([-res, self.idx, other.idx])
        cnf.append([res, -self.idx, -other.idx])

    def __sub__(self, other):
        pass

def half_adder(a,b):
    s = a ^ b
    c = a & b
    return s, c
def full_adder(a,b,c):
    s = a ^ b ^ c
    c = (a & b) | (a & c) | (b & c)
    return s, c
class BV():
    def __init__(self, name, n, vs=None):
        self.name = name
        self.n = n
        if vs:
            self.vars = vs
        else:
            self.vars = [Var(f"{name}_{i}") for i in range(n)]
    def __getitem__(self, i):
        return self.vars[i]
    def __add__(self, other):
        pass






# class Var(): name, counter, cnf # a global cnf?
x = Var("x")
y = Var("y")
x.idx
y.idx
Var("z").idx

BV = list[Var]

def bvadd(self, a, b):
    c = 0
    cnf = []
    for i in range(len(a)):

        s = a[i] ^ b[i] ^ c
        c = (a[i] & b[i]) | (a[i] & c) | (b[i] & c)
        cnf.append(s)
    res.append(c)
    return c, cnf
class Solver():
    def bvadd(self, a, b):
        c = 0
        res = []
        for i in range(len(a)):
            s = a[i] ^ b[i] ^ c
            c = (a[i] & b[i]) | (a[i] & c) | (b[i] & c)
            res.append(s)
        res.append(c)
        return res
    def bvsub(self, a,b):

    def bveq(self, a,b):



2

# Bits and Bobbles

I would like to take the different variations of sat solving and show how to move between them via defunctionalization and the like


[The silent revolution of sat](https://news.ycombinator.com/item?id=36079115#36081904)

[creusat](https://sarsko.github.io/_pages/SarekSkot%C3%A5m_thesis.pdf) <https://github.com/sarsko/CreuSAT>

<https://fmv.jku.at/fleury/papers/Fleury-thesis.pdf>

<https://github.com/marijnheule/microsat>

<https://www-cs-faculty.stanford.edu/~knuth/programs.html> knuth sat solvers and others

- <https://www.youtube.com/watch?v=II2RhzwYszQ&t=14916s&ab_channel=SimonsInstitute>
- <https://github.com/arminbiere/satch>


3-sat solver. Requiring all clauses be in 3sat form may have certain conveniences.

CPS style backtracking ---> sequent proof objects


```python
from typing import NamedTuple

Literal = NamedTuple('Literal', [('idx', int), ('polarity', bool)])
Clause = NamedTuple('Clause', [('literals', list)])
Formula = NamedTuple('Formula', [('clauses', list[Clause]), ('num_vars', int)])

Assignment = NamedTuple('Assignment', [('values', list[bool])])
Pasn = NamedTuple('Pasn', [('assignment', Assignment), ('satisfiable', bool)])

```

# SMT

<https://homepage.cs.uiowa.edu/~tinelli/papers/NieOT-JACM-06.pdf> Solving SAT and SAT Modulo Theories: from an
Abstract Davis-Putnam-Logemann-Loveland
Procedure to DPLL(T)

<https://citeseerx.ist.psu.edu/document?repid=rep1&type=pdf&doi=45036ae8e05d71ea39253f863cfa270b6cdd9aa2> simplify paper

User theories. <https://microsoft.github.io/z3guide/programming/Example%20Programs/User%20Propagator/>

<https://cs.stanford.edu/~preiner/talks/Preiner-Shonan23.pdf> ipasir-up

<https://github.com/Z3Prover/z3/blob/master/src/smt/smt_theory.h>

```python
class Theory():
    def mk_var(self, enode):
        pass
    def getvar(self, enode):
        pass
    def propagate(self):
        pass
    def push(self):
        pass
    def pop(self, n):
        pass
    def assign(self, bvar, sign):
        pass
    def new_eq(self, v1, v2):
        pass
    def new_diseq(self, v1,v2):
        pass
    def validate_model(self, mode):
        pass
    # model generate
    def 
    # model check
    def m

```

- <https://github.com/cvc5/cvc5/blob/main/src/theory/theory.h>
- <https://github.com/cvc5/cvc5/blob/main/src/theory/output_channel.h>
- <https://ocamlpro.github.io/alt-ergo/latest/API/alt-ergo-lib/AltErgoLib/Theory/module-type-S/index.html>


### z3 as a theory

One of the easiest ways of getting a theory solve is to just use z3 as one.
If you assert only theory atoms, basically it's a theory.
You can propagate via many queries.

We could also bolt z3 to cvc5 as an abomination.
Or bolt kissat to z3.
Or z3 that doesn't know what the other hand do-eth

```python
from z3 import *

boolmap = {}





```

### Intercommunicating cvc5 and z3

### dreal = z3 + flint (?)

SMT modulo convex

Could I find piles of interesting little SMT theories to prototype up?
